In [8]:
import deeplay as dl
import torch.nn as nn

# Styles

Styles are a collection of configurations that can be applied to a deeplay module. Different modules will have different styles available. You can check the available styles using the `available_styles` method. This method can be called either on the class or an instance of the class.

In [3]:
dl.Conv2dBlock.available_styles()

['residual',
 'spatial_self_attention',
 'spatial_cross_attention',
 'spatial_transformer',
 'resnet',
 'resnet18_input']

We can apply a style using the `style` method. The `style` method takes the name of the style as an argument. 

In [5]:
block = dl.Conv2dBlock(3, 64).style("residual")

It is fine to chain `style` with other configuration methods. However, it is recommended to apply the style first. Most styles assume that the module is in an unstyled state. Applying a style after other configurations may lead to unexpected results.

In [6]:
block = dl.Conv2dBlock(3, 64).style("residual").strided(2)
block

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=1, stride=2, padding=0)
    (activation): Layer[Identity]()
  )
  (blocks): Sequential(
    (0): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=0)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
    (1): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
  )
  (shortcut_end): Add()
)

Some styles may have additional parameters. These parameters can be passed as keyword arguments to the `style` method. Your editor should show you the available parameters as well as a documentation string for the style.

In [7]:
block = dl.Conv2dBlock(3, 64).style("residual", order="lala|")
block

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=1, stride=1, padding=0)
    (activation): Layer[Identity]()
  )
  (blocks): Sequential(
    (0): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=0)
      (activation): Layer[ReLU]()
    )
    (1): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0)
      (activation): Layer[ReLU]()
    )
  )
  (shortcut_end): Add()
)

## Extending Styles

You can easily create your own styles. This is useful to apply a set of configurations to multiple modules. To create a style, you simply need to use the `@class.register_style` decorator. The name of the function will be the name of the style. The function should take a module as the first argument, and any additional parameters as keyword arguments.

In [16]:
@dl.LinearBlock.register_style
def my_linear(block: dl.LinearBlock, bias_in_shortcut=False):
    block.activated(nn.GELU)
    block.normalized(nn.BatchNorm1d)
    block.shortcut(shortcut=dl.Layer(nn.Linear, block.in_features, block.out_features, bias=bias_in_shortcut))
    return block

dl.LinearBlock.available_styles()

['my_linear']

In [15]:
block = dl.LinearBlock(3, 64).style("my_linear")
block_with_bias = dl.LinearBlock(3, 64).style("my_linear", bias_in_shortcut=True)
block, block_with_bias

(LinearBlock(
   (shortcut_start): Layer[Linear](in_features=3, out_features=64, bias=False)
   (layer): Layer[Linear](in_features=3, out_features=64, bias=True)
   (activation): Layer[GELU]()
   (normalization): Layer[BatchNorm1d](num_features=64)
   (shortcut_end): Add()
 ),
 LinearBlock(
   (shortcut_start): Layer[Linear](in_features=3, out_features=64, bias=True)
   (layer): Layer[Linear](in_features=3, out_features=64, bias=True)
   (activation): Layer[GELU]()
   (normalization): Layer[BatchNorm1d](num_features=64)
   (shortcut_end): Add()
 ))